In [ ]:
'''
匯入套件
'''
# 操作 browser 的 API
from selenium import webdriver

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 強制等待 (執行期間休息一下)
from time import sleep

# 整理 json 使用的工具
import json

# 執行 command 的時候用的
import os

# 子處理程序，用來取代 os.system 的功能
import subprocess

# 取得隨機數
import random

# 啟動瀏覽器工具的選項
my_options = webdriver.ChromeOptions()
my_options.add_argument("--start-maximized") #最大化視窗

# 使用 Chrome 的 WebDriver
driver = webdriver.Chrome(
    options = my_options,
)

# 建立儲存圖片、影片的資料夾
folderPath = 'wikiart'
if not os.path.exists(folderPath):
    os.makedirs(folderPath)

# 放置爬取的資料
set_data = set()

In [ ]:
def visit():
    driver.get("https://www.wikiart.org/")

# 滾動頁面
def scroll():
    innerHeight = 0
    offset = 0
    count = 0
    limit = 2
    move = 0
    
    # 在捲動到沒有元素動態產生前，持續捲動
    while count <= limit:
        # 每次移動的距離
        offset += 800

        # 捲軸往下滑動
        driver.execute_script(f'''
            window.scrollTo({{
                top: {offset}, 
                behavior: 'smooth' 
            }});
        ''')

        # 每次捲動完，就執行一次載入更多的動作
        if move % 5 == 0:
            load_more()

        # 每次捲動完，就執行一次解析的動作
        parse()

        # 每次捲動完，就執行一次儲存的動作
        save()
        
        # (重要)強制等待，此時若有新元素生成，瀏覽器內部高度會自動增加
        sleep(random.randint(2, 3))
        
        # 透過執行 js 語法來取得捲動後的當前總高度
        innerHeight = driver.execute_script(
            'return document.documentElement.scrollHeight;'
        )

        # 每次捲動完，就將移動次數加 1
        move += 1

        print(f"count: {count}, offset: {offset}, innerHeight: {innerHeight}")
        
        # 經過計算，如果滾動距離(offset)大於等於視窗內部總高度(innerHeight)，代表已經到底了
        if offset >= innerHeight:
            count += 1
            
# 載入更多圖片
def load_more():
    try:
        css_selector_load_more = 'a.masonry-load-more-button[ng-show="canLoadMore()"]'
        if len(driver.find_elements(By.CSS_SELECTOR, css_selector_load_more)) > 0:
            WebDriverWait(driver, 1).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, css_selector_load_more))
            )
            driver.find_element(By.CSS_SELECTOR, css_selector_load_more).click()
    except TimeoutException:
        print('Load More 按鈕失效')

# 解析資料
def parse():
    global set_data
    try:
        elements = driver.find_elements(By.CSS_SELECTOR, 'img[src^=https\:\/\/upload]')
        for element in elements:
            # 取得圖片連結
            src = element.get_attribute('src')
            set_data.add(src)
    except:
        pass

# 儲存資料
def save():
    global set_data
    with open(f'{folderPath}/data.json', 'w', encoding='utf-8') as f:
        f.write(json.dumps(list(set_data), indent=4, ensure_ascii=False))

In [ ]:
if __name__ == '__main__':
    visit()
    scroll()

In [ ]:
# 關閉瀏覽器
driver.quit()

In [ ]:
# 下載圖片
with open(f'{folderPath}/data.json', 'r', encoding='utf-8') as f:
    # 讀取 json 檔案，變成 list
    list_data = json.loads(f.read())

    # 逐一下載圖片
    for i, src in enumerate(list_data):
        # 練習期間，下載幾張就好
        if i == 10:
            break

        # 清除圖片網址的 !PinterestSmall.jpg，取得最高畫質的圖片連結
        src = src.replace('!PinterestSmall.jpg', '')

        # 取得圖片檔名
        file_name = src.split('/')[-1]
        
        print(f'第 {i+1} 張：{src}')
        
        # 下載圖片
        std = subprocess.run(['curl', src, '-o', f'{folderPath}/{file_name}'])
        if std.returncode == 0:
            print(f'{file_name} 下載成功！')

        # 每下載一張就強制等待
        sleep(1)